In [1]:
import json
import pandas as pd



In [2]:
data = []

with open('../../DATA/JSON/all_data.json') as file:
    data = json.load(file)

In [3]:
final_agency_name_mapping = pd.read_csv('../13_Merge_Table/Final_Agency_Name_Mapping.csv')
final_agency_name_mapping

,Agency_Name,Mapped_Name,Country
0,100 Anniversary Chulalongkorn University,100 Anniversary Chulalongkorn University,Thailand
1,100-year-old Chulalongkorn University,100-year-old Chulalongkorn University,Thailand
2,100th Anniversary Chulalongkorn University,100th Anniversary Chulalongkorn University,Thailand
3,100th Anniversary Chulalongkorn University Fun...,100th Anniversary Chulalongkorn University,Thailand
4,100th Anniversary Chulalongkorn University for...,100th Anniversary Chulalongkorn University,Thailand
...,...,...,...
8329,trustee of Allergy UK,trustee of Allergy UK,United Kingdom
8330,world class university,Thailand National Organization,Thailand
8331,Åbo Akademi University,Åbo Akademi University,Finland
8332,Österreichische Agentur für Internationale Mob...,Österreichische Agentur für Internationale Mob...,Austria


In [4]:
funding_field = []
funding_field_agency_one_row = []

for i in data:
    eid = i['abstracts-retrieval-response']['coredata']['eid']
    areas = i['abstracts-retrieval-response']['subject-areas']['subject-area']
    item = i['abstracts-retrieval-response']['item']
    year = i['abstracts-retrieval-response']['coredata']['prism:coverDate'].split('-')[0]
    try:
        if (item['xocs:meta'] != None):
            if (item['xocs:meta']['xocs:funding-list'] != None):
                fundings = item['xocs:meta']['xocs:funding-list']['xocs:funding']

                for funding in fundings:
                    funding_agency_name = funding.get('xocs:funding-agency-matched-string', None)
                    funding_agency_country = funding.get('xocs:funding-agency-country', None)
                    for area in areas:
                        funding_field.append([eid, funding_agency_name, funding_agency_country, area['@code'], year])
                
    except:
        pass


for i in data:
    eid = i['abstracts-retrieval-response']['coredata']['eid']
    areas = i['abstracts-retrieval-response']['subject-areas']['subject-area']
    item = i['abstracts-retrieval-response']['item']
    year = i['abstracts-retrieval-response']['coredata']['prism:coverDate'].split('-')[0]
    try:
        if (item['xocs:meta'] != None):
            if (item['xocs:meta']['xocs:funding-list'] != None):
                fundings = item['xocs:meta']['xocs:funding-list']['xocs:funding']
                funding_string = ''
                funding_list = set()
                for funding in fundings:
                    funding_agency_name = funding.get('xocs:funding-agency-matched-string', None)
                    test_agency_name = pd.DataFrame([funding_agency_name], columns=['Agency_Name'])
                    test_agency_name = test_agency_name.merge(final_agency_name_mapping, on='Agency_Name', how='left')
                    mapped_name = (test_agency_name['Mapped_Name'].iloc[0])
                    if(mapped_name == None):
                        print(111)
                    funding_list.add(mapped_name)
                funding_list = list(funding_list)
                funding_list.sort()
                funding_string = ' '.join(funding_list)
                for area in areas:
                    funding_field_agency_one_row.append([eid, funding_string, area['@code'], year])
                
    except:
        for area in areas:
            funding_field_agency_one_row.append([eid, None, area['@code'], year])       
        pass


In [5]:
funding_organization_interest_df = pd.DataFrame(funding_field, columns=['Eid', 'Funding_Agency_Name', 'Funding_Agency_Country_Link', 'Subject_Field', 'Year'])
funding_field_agency_one_row_df = pd.DataFrame(funding_field_agency_one_row, columns=['Eid', 'Funding_Agency_Name', 'Subject_Field', 'Year'])

In [6]:
funding_organization_interest_df = funding_organization_interest_df.reset_index().drop(columns='index')
funding_field_agency_one_row_df = funding_field_agency_one_row_df.reset_index().drop(columns='index')

In [7]:
funding_organization_interest_df['Funding_Agency_Country_Link'] = funding_organization_interest_df['Funding_Agency_Country_Link'].apply(lambda x : None if x == None else x + '/' if x[-1] != '/' else x)

In [8]:
funding_field_agency_one_row_df['Eid'].nunique()

20216

In [9]:
funding_organization_interest_df

,Eid,Funding_Agency_Name,Funding_Agency_Country_Link,Subject_Field,Year
0,2-s2.0-85053164279,Ratchadapisek Sompoch Endowment Fund,None,2500,2018
1,2-s2.0-85053164279,Ratchadapisek Sompoch Endowment Fund,None,3104,2018
2,2-s2.0-85053164279,Surface Coatings Technology for Metals and Mat...,None,2500,2018
3,2-s2.0-85053164279,Surface Coatings Technology for Metals and Mat...,None,3104,2018
4,2-s2.0-85053164279,NSFC,http://sws.geonames.org/1814991/,2500,2018
...,...,...,...,...,...
151745,2-s2.0-85165609857,Thailand Science Research and Innovation Fund,http://sws.geonames.org/1605651/,1106,2023
151746,2-s2.0-85165609857,Thailand Science Research and Innovation Fund,http://sws.geonames.org/1605651/,1314,2023
151747,2-s2.0-85165609857,Thailand Science Research and Innovation Fund,http://sws.geonames.org/1605651/,1103,2023
151748,2-s2.0-85165609857,Thailand Science Research and Innovation Fund,http://sws.geonames.org/1605651/,1311,2023


In [10]:
funding_organization_interest_df.drop_duplicates()


,Eid,Funding_Agency_Name,Funding_Agency_Country_Link,Subject_Field,Year
0,2-s2.0-85053164279,Ratchadapisek Sompoch Endowment Fund,None,2500,2018
1,2-s2.0-85053164279,Ratchadapisek Sompoch Endowment Fund,None,3104,2018
2,2-s2.0-85053164279,Surface Coatings Technology for Metals and Mat...,None,2500,2018
3,2-s2.0-85053164279,Surface Coatings Technology for Metals and Mat...,None,3104,2018
4,2-s2.0-85053164279,NSFC,http://sws.geonames.org/1814991/,2500,2018
...,...,...,...,...,...
151745,2-s2.0-85165609857,Thailand Science Research and Innovation Fund,http://sws.geonames.org/1605651/,1106,2023
151746,2-s2.0-85165609857,Thailand Science Research and Innovation Fund,http://sws.geonames.org/1605651/,1314,2023
151747,2-s2.0-85165609857,Thailand Science Research and Innovation Fund,http://sws.geonames.org/1605651/,1103,2023
151748,2-s2.0-85165609857,Thailand Science Research and Innovation Fund,http://sws.geonames.org/1605651/,1311,2023


In [11]:
funding_organization_interest_df['Funding_Agency_Country_Code'] = funding_organization_interest_df['Funding_Agency_Country_Link'].apply(lambda x : None if x == None else  x.split('/')[-2])

In [12]:
organization_country_code = funding_organization_interest_df['Funding_Agency_Country_Link'].apply(lambda x : None if x == None else x.split('/')[-2]).drop_duplicates().to_list()

In [13]:
funding_organization_interest_df = funding_organization_interest_df.drop(columns='Funding_Agency_Country_Link', axis=1)

In [14]:
funding_organization_interest_df

,Eid,Funding_Agency_Name,Subject_Field,Year,Funding_Agency_Country_Code
0,2-s2.0-85053164279,Ratchadapisek Sompoch Endowment Fund,2500,2018,None
1,2-s2.0-85053164279,Ratchadapisek Sompoch Endowment Fund,3104,2018,None
2,2-s2.0-85053164279,Surface Coatings Technology for Metals and Mat...,2500,2018,None
3,2-s2.0-85053164279,Surface Coatings Technology for Metals and Mat...,3104,2018,None
4,2-s2.0-85053164279,NSFC,2500,2018,1814991
...,...,...,...,...,...
151745,2-s2.0-85165609857,Thailand Science Research and Innovation Fund,1106,2023,1605651
151746,2-s2.0-85165609857,Thailand Science Research and Innovation Fund,1314,2023,1605651
151747,2-s2.0-85165609857,Thailand Science Research and Innovation Fund,1103,2023,1605651
151748,2-s2.0-85165609857,Thailand Science Research and Innovation Fund,1311,2023,1605651


In [15]:
funding_organization_interest_df['Funding_Agency_Name'].nunique()

10883

In [16]:
funding_organization_interest_df['Funding_Agency_Country_Code'].drop_duplicates().dropna().to_csv('country_code_for_scrape.csv', index=False)

In [17]:
funding_organization_interest_df.to_csv('funding_organization_interest.csv', index=False)

In [19]:
funding_field_agency_one_row_df

,Eid,Funding_Agency_Name,Subject_Field,Year
0,2-s2.0-85053164279,Chulalongkorn University NSFC National Science...,2500,2018
1,2-s2.0-85053164279,Chulalongkorn University NSFC National Science...,3104,2018
2,2-s2.0-85049101440,None,1403,2018
3,2-s2.0-85049101440,None,2000,2018
4,2-s2.0-85054140369,"Faculty of Medicine, Khon Kaen University Raja...",3612,2018
...,...,...,...,...
50059,2-s2.0-85165609857,Chulalongkorn University Thailand National Org...,1311,2023
50060,2-s2.0-85165609857,Chulalongkorn University Thailand National Org...,3400,2023
50061,2-s2.0-85164405594,None,1203,2023
50062,2-s2.0-85164405594,None,3310,2023


In [18]:
funding_field_agency_one_row_df.to_csv('funding_field_agency_one_row.csv', index=False)

In [20]:
funding_field_agency_one_row_df.isna().sum()

Eid                        0
Funding_Agency_Name    30531
Subject_Field              0
Year                       0
dtype: int64